# Example using the FraudDetector SDK 
Demonstration of how the SDK simplifies using the AWS Fraud Detector service.

In [2]:
# Set authentiation to AWS via temporary access tokens
import json
import os
from getpass import getpass

*Either* setup authentication to AWS cloud environment before starting this notebook, *or* use the next section to enter access keys and tokens for AWS cloud account to run this in.

In [3]:
# Optional section to set access keys and tokens for AWS cloud account access:
ACCESS_KEY = getpass("Enter the AWS Access Key:")
SECRET_KEY =  getpass("Enter the AWS Secret Key:")
SESSION_TOKEN = getpass("Enter the AWS Session Token to use:")

os.environ['AWS_ACCESS_KEY_ID'] = ACCESS_KEY
os.environ['AWS_SECRET_ACCESS_KEY'] = SECRET_KEY
os.environ['AWS_SESSION_TOKEN'] = SESSION_TOKEN

Enter the AWS Access Key:········
Enter the AWS Secret Key:········
Enter the AWS Session Token to use:········


In [4]:
# set the import path relative to the notebook examples location
import sys
sys.path.append("../")

### FraudDetector SDK Imports

In [5]:
import pandas as pd
from frauddetector import frauddetector, profiler

# Profile Data with `Profiler()`; generate Data Schema, Variables Definition, Label Definitions
A representative set of data needs to be loaded into a Pandas data-frame for the profiler to generate the FraudDetector definitions from.

In [6]:
profiler = profiler.Profiler()
df = pd.read_csv("training_data/registration_data_20K_minimum.csv")
data_schema, variables, labels = profiler.get_frauddetector_inputs(data=df)

In [7]:
variables

[{'name': 'ip_address',
  'variableType': 'IP_ADDRESS',
  'dataType': 'STRING',
  'defaultValue': 'unknown'},
 {'name': 'email_address',
  'variableType': 'EMAIL_ADDRESS',
  'dataType': 'STRING',
  'defaultValue': 'unknown'}]

In [8]:
labels

[{'name': 'legit'}, {'name': 'fraud'}]

# Instantiate a `FraudDetector()` with variables and labels

In [9]:
detector = frauddetector.FraudDetector(
    entity_type="registration",
    event_type="user-registration",
    detector_name="registration-project",
    model_name="registration_model",
    model_version="1.00",
    model_type="ONLINE_FRAUD_INSIGHTS",
    region='eu-west-1',
    detector_version="1"
    )

# Train a Model - `fit()` stage
Supply an IAM role ARN as well as the training data location in S3.  The role should have AmazonFraudDetectorFullAccessPolicy attached to it.

In [19]:
# https://docs.aws.amazon.com/frauddetector/latest/ug/security-iam.html
role_arn="arn:aws:iam::9999999999:role/MyFraudDetectorRole"

In [22]:
detector.fit(data_schema=data_schema
         , data_location="s3://<my-s3-bucket>/training/registration_data_20K_minimum.csv"             
         , role=role_arn
         , variables=variables
         , labels=labels)

create_entity_type: entity registration already exists, skipping
create_labels: label legit already exists, skipping
create_labels: label fraud already exists, skipping
create_variables: variable ip_address already exists, skipping
create_variables: variable email_address already exists, skipping
create_event_type: event user-registration already exists, skipping


{'modelId': 'registration_model',
 'modelType': 'ONLINE_FRAUD_INSIGHTS',
 'modelVersionNumber': '1.0',
 'status': 'TRAINING_IN_PROGRESS',
 'ResponseMetadata': {'RequestId': 'c1655b8b-eaf9-46e0-ae6f-724f04d1cd5c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 Dec 2021 16:17:51 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '127',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c1655b8b-eaf9-46e0-ae6f-724f04d1cd5c'},
  'RetryAttempts': 0}}

# Create a Detector - `activate()` stage
Provide a list of outcomes to create an active model associted with FraudDetector "outcomes" ready for FD rules to be associated with the outcomes.

In [11]:
# get the model status - should be TRAINING_COMPLETE before starting compile stage.
print(detector.model_status)

TRAINING_COMPLETE


In [12]:
print(detector.model_status)

TRAINING_COMPLETE


In [13]:
print(detector.model_version)

1.00


In [14]:
outcomes = [("review_outcome", "Start a review process workflow"),
                  ("verify_outcome", "Sideline event for review"),
                  ("approve_outcome", "Approve the event")]

In [15]:
detector.activate(outcomes_list=outcomes)

In [16]:
# get the model status - should be ACTIVE after compile stage (will be ACTIVATE_IN_PROGRESS for some time)
print(detector.model_status)

ACTIVATE_IN_PROGRESS


In [19]:
print(detector.model_status)

ACTIVE


# Deploy a Fraud Detector - `deploy()` stage
Provide a list of rules that map to rule-outcomes to deploy the detector-version ready for fraud predicition actions.
Each rule item in the list is a dictionary of
```
{'ruleId': 'name_of_rule',
'expression': 'rule_expression_for_evaluating_rule', 
'outcomes': [list_of, outcomes_for, matching_rule]
}
```
See documentation here: https://docs.aws.amazon.com/frauddetector/latest/ug/rule-language-reference.html

In [20]:
print(detector.model_name)

registration_model


The model name with `_insightscore` appended to it is used to reference the output value from the model

In [21]:
rules = [{'ruleId': 'high_fraud_risk',
                       'expression': '$registration_model_insightscore > 900',
                       'outcomes': ['verify_outcome']
                      },
        {'ruleId': 'low_fraud_risk',
                       'expression': '$registration_model_insightscore <= 900 and $registration_model_insightscore > 700',
                       'outcomes': ['review_outcome']
                      },
        {'ruleId': 'no_fraud_risk',
                       'expression': '$registration_model_insightscore <= 700',
                       'outcomes': ['approve_outcome']
                      } 
        ]

In [22]:
response = detector.deploy(rules_list=rules)

In [23]:
print(response)

{'detectorId': 'registration-project', 'detectorVersionId': '1', 'status': 'DRAFT', 'ResponseMetadata': {'RequestId': '03c5ed13-3090-4dbe-9af5-8045059a710f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 01 Dec 2021 18:02:55 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '78', 'connection': 'keep-alive', 'x-amzn-requestid': '03c5ed13-3090-4dbe-9af5-8045059a710f'}, 'RetryAttempts': 0}}


# Get Predictions - `predict()` stage
Use the `predict()` or `batch_predict()` methods to predict for a single event, passed in as a dictionary, or a batch of events passed in as a dataframe.

**Example**: Single event that triggers the `low_fraud_risk` rule resulting in a `review_outcome`

In [24]:
event_variables = {
            'email_address' : 'johndoe@exampledomain.com',
            'ip_address' : '1.2.3.4'
            }

detector.predict('2021-11-13T12:18:21Z',event_variables)

{'registration_model_insightscore': 861.0,
 'ruleResults': [{'ruleId': 'low_fraud_risk', 'outcomes': ['review_outcome']}]}

**Example** Single event that triggers the `no_fraud_risk` rule resulting in an `approve_outcome`

In [25]:
event_variables = {
            'email_address' : 'johndoe@gmail.com',
            'ip_address' : '82:24:61:42'
        }
detector.predict('2021-11-14T12:18:21Z',event_variables)

{'registration_model_insightscore': 355.0,
 'ruleResults': [{'ruleId': 'no_fraud_risk', 'outcomes': ['approve_outcome']}]}

# Destroy resources

In [ ]:
detector.delete_detector()

In [ ]:
detector.rules

In [ ]:
rule_ids = [r['ruleId'] for r in detector.rules]

In [ ]:
print(rule_ids)

In [ ]:
detector.delete_rules(detector.rules)

In [ ]:
detector.rules

In [ ]:
detector.delete_detector_version()

In [ ]:
#de-activate the model
detector.set_model_version_inactive()

In [ ]:
print(detector.model_name)

In [ ]:
# get the model status - should be INACTIVE or TRAINING_COMPLETE before deleting it
print(detector.model_status)

In [ ]:
#delete the model
detector.delete_model()

In [ ]:
detector.delete_detector_version()

In [ ]:
detector.fd.delete_detector_version(detectorId=detector.detector_name, detectorVersionId=detector.detector_version)